In [51]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
from PIL import Image
import glob
import re
from random import *
from os import listdir
from pickle import dump
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import numpy as np
from keras.models import Model
from keras.models import load_model
from numpy import argmax
import cv2
import matplotlib.pyplot as plt
from keras.layers import Bidirectional
import cv2
from keras import optimizers
import networkx as nx

# ***Creating Captions***

In [0]:
image_list = {}                                                                      #dictionary for image and caption
captions = []                                                                        # captions array
for filename in glob.glob('/content/drive/My Drive/Colab_Notebooks/NewImage/*.jpg'):      #assuming gif
    img=Image.open(filename)                                                         #loading the image                                                                #the take the filename 
    Unique = int(re.sub("\D", "", filename ))                                        #removing all the letters to create unique ID
    print(Unique)
    if  Unique <310 :
      caption1 = "pan on stove";                                             #defining captions
      caption2 = "stove below pan";   
      captions = [caption1,caption2]                                                 # assigning captions array
      image_list.update({Unique:[img,captions]})                                     # creating the dictionary of images with captions and uniques key
    elif ( Unique <633 and Unique >= 310):
      caption1 = "egg on hand and pan on stove";
      caption2 = "pan on stove and egg on hand";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif ( Unique <1033 and Unique >= 633 ):
      caption1 = "drop egg on pan and hand over pan";
      caption2 = "hand over pan and egg drop on pan";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif ( Unique <1121  and Unique >= 1033):
      caption1 = "egg on pan and hand over pan";
      caption2 = "hand over pan and egg on pan";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif (Unique <1289 and Unique >=1121):
      caption1 = "egg on pan";
      caption2 = "egg over pan"; 
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})

    


In [57]:
image_list = {}                                                                      #dictionary for image and caption
captions = []                                                                        # captions array
for filename in glob.glob('/content/drive/My Drive/Colab_Notebooks/testImg/*.jpg'):      #assuming gif
    img=Image.open(filename)                                                         #loading the image                                                                #the take the filename 
    Unique = int(re.sub("\D", "", filename ))                                        #removing all the letters to create unique ID
    print(Unique)
    if  Unique >312 :
      caption1 = "pan on stove";                                             #defining captions
      caption2 = "stove below pan";   
      captions = [caption1,caption2]                                                 # assigning captions array
      image_list.update({Unique:[img,captions]})                                     # creating the dictionary of images with captions and uniques key
    elif ( Unique <40 and Unique >= 0):
      caption1 = "egg on hand and pan on stove";
      caption2 = "pan on stove and egg on hand";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif ( Unique <116 and Unique >= 40 ):
      caption1 = "drop egg on pan and hand over pan";
      caption2 = "hand over pan and egg drop on pan";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif ( Unique <178  and Unique >= 116):
      caption1 = "egg on pan and hand over pan";
      caption2 = "hand over pan and egg on pan";
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})
    elif (Unique <311 and Unique >=178):
      caption1 = "egg on pan";
      caption2 = "egg over pan"; 
      captions = [caption1,caption2]
      image_list.update({Unique:[img,captions]})


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# ***Helper Funnctions***

In [0]:
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list[1]:
			lines.append(str(key) + ' ' + desc)
	data = '\n'.join(lines)
	file = open('/content/drive/My Drive/Colab_Notebooks/'+filename, 'w')
	file.write(data)
	file.close()

# ***Extracting Features from image***

In [0]:
def extract_features(directory):
	# load the model
	model = VGG19()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()
	for image_id,image in directory.items():
		image = image[0].resize((224,224), Image.ANTIALIAS)
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# store feature
		features[image_id] = feature
		print('>%s' % image_id)
	return features

# ***Load description and features from file***

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# load clean descriptions into memory
def load_clean_descriptions(filename):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		# create list
		if image_id not in descriptions:
			descriptions[image_id] = list()
		# wrap description in tokens
		desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
		# store
		descriptions[image_id].append(desc)
	return descriptions

In [0]:
def load_photo_features(filename):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	#features = {k: all_features[k] for k in dataset}
	return all_features

# ***Tokenizing Descriptions***

In [0]:
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 


# ***Creating sequence for LSTM***

In [0]:
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        
				X1.append(photos[int(key)][0])
				X2.append(in_seq)
				y.append(out_seq)
	return np.array(X1), np.array(X2), np.array(y)


# ***Building Graph***

In [0]:
def creating_a_graphs(data,preposition):
  #creating a  dictionary of the preposition, conecting word and noun
  data_dict = {}
  dictionary_of_graphs = {}
  for x in range(len(data)):
    New_data = []
    if(data[x] in preposition):
      New_data.append(data[x-1])
      New_data.append(data[x+1])
      data_dict[data[x]] = New_data


  #creating a graph with nouns as nodes and preposition as edges
  G=nx.DiGraph()
  for key,value in data_dict.items():
    G.add_nodes_from(value)
    G.add_edge(value[0],value[1])
    G.edges[value[0],value[1]]["preposition"] = key

  #dictionary_of_graphs[time_stamp] = G


  #plotting the grap with nodes and edges as tags
  node_color = [G.degree(v) for v in G]  
  node_size = 1100
  edge_width = 0.7
  
  pos = nx.spring_layout(G)

  nx.draw_networkx(G, pos,node_size = node_size,  
                   node_color = node_color, alpha = 0.7, 
                   with_labels = True, width = edge_width, 
                   edge_color ='.4', cmap = plt.cm.Blues) 

  node_labels = nx.get_node_attributes(G,'preposition')
  nx.draw_networkx_labels(G, pos, labels = node_labels)
  edge_labels = nx.get_edge_attributes(G,'preposition')
  nx.draw_networkx_edge_labels(G, pos, labels = edge_labels)

  plt.axis('off') 
  plt.tight_layout()
  plt.show()
  
  return  G

# ***LSTM MODEL***

In [0]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = Bidirectional(LSTM(128))(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer= optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False))
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

# ***Extracting feature and description***

In [0]:
save_descriptions(image_list,"desciption_new_train_with_preposition_connecting_words_testImg.txt")

In [0]:
features = extract_features(image_list)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('/content/drive/My Drive/Colab_Notebooks/features_new_train_VGG19_for_new_text_testImg.pkl', 'wb'))

# ***Making train set***

In [54]:
train_descriptions = load_clean_descriptions('/content/drive/My Drive/Colab_Notebooks/desciption_new_train_with_preposition_connecting_words.txt')
print('Descriptions: train=%d' % len(train_descriptions))
#photo features
train_features = load_photo_features('/content/drive/My Drive/Colab_Notebooks/features_new_train_VGG19_for_new_text_NewImg.pkl')
print('Photos: train=%d' % len(train_features))
print(train_features[0][0])

Descriptions: train=1289
Photos: train=1289
[0.21275729 0.         2.9594917  ... 0.         0.         1.2690203 ]


In [55]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)


Vocabulary Size: 12
Description Length: 10


In [0]:
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

# ***Making Validation Set***

In [59]:
train_descriptions_dev = load_clean_descriptions('/content/drive/My Drive/Colab_Notebooks/desciption_new_train_with_preposition_connecting_words_testImg.txt')
print('Descriptions: train=%d' % len(train_descriptions_dev))
#photo features
train_features_dev = load_photo_features('/content/drive/My Drive/Colab_Notebooks/features_new_train_VGG19_for_new_text_testImg.pkl')
print('Photos: train=%d' % len(train_features_dev))
print(train_features[0][0])

Descriptions: train=322
Photos: train=322
[0.21275729 0.         2.9594917  ... 0.         0.         1.2690203 ]


In [0]:
X1test, X2test, ytest = create_sequences(tokenizer, max_length, train_descriptions_dev, train_features_dev, vocab_size)

# ***Training Model***

In [61]:


# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = '/content/drive/My Drive/Colab_Notebooks/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}_VGG19.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 10, 256)      3072        input_9[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 4096)         0           input_8[0][0]                    
____________________________________________________________________________________________

KeyboardInterrupt: ignored

# ***Generating Captions***

In [0]:
def extract_features_test(filename):
	# load the model
	model = VGG19()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

In [0]:
def extract_features_test_video(image):
	# load the model
	model = VGG19()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	image = cv2.resize(image, (224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

In [0]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [0]:
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

In [35]:
max_length = 10
Array_of_description = None
dictionary_of_graphs = {}
preposition = ["below","above","and","on","over"]
counter = 0
# load the model
model = load_model('/content/drive/My Drive/Colab_Notebooks/model-ep004-loss0.159-val_loss0.308_VGG19.h5')
# load and prepare the photograph
for x in range(0,6):
  #picRand = str(randint(0,20))
  image_position = 0
  Pictures =Image.open('/content/drive/My Drive/Colab_Notebooks/testImg/'+str(image_position)+'.jpg')
#   plt.imshow(Pictures)
#   plt.show()
  
  photo = extract_features_test('/content/drive/My Drive/Colab_Notebooks/testImg/'+str(image_position)+'.jpg')
  # generate description
  
  description = generate_desc(model, tokenizer, photo, max_length)
  data = description.split()
  
  if(description != Array_of_description ):
    dictionary_of_graphs[counter] = creating_a_graphs(data,preposition)
    counter += 1
    Array_of_description = description
    
  image_position +=53
  print(dictionary_of_graphs)
  print(description)


198066176/574710816 [=========>....................] - ETA: 24s

KeyboardInterrupt: ignored

# ***Translating the pictures for a dictionary***

In [0]:

max_length = 10
# load the model
model = load_model('/content/drive/My Drive/Colab_Notebooks/model-ep007-loss0.118-val_loss0.369_VGG16.h5')

Map_of_pic = {}

temp = None

seq = 0

vidcap = cv2.VideoCapture('/content/drive/My Drive/Colab_Notebooks/VID3.mp4')
fps = vidcap.get(cv2.CAP_PROP_FPS)
success,image = vidcap.read()
count = 0

while success:
  # dividing the video into franes
  success,image = vidcap.read()
  # extracting features from photo
  photo = extract_features_test_video(image)
  print('Read a new frame: ', success)
  count += 1
  # calculating time per frame
  time = count/fps
  # generating caption
  description = generate_desc(model, tokenizer, photo, max_length)
  print(description)
  #
  if description != temp and temp == None:
      Map_of_pic[seq] = [description,time]
      seq = seq + 1  
  
  temp = description

print(Map_of_pic)
cam.release() 
cv2.destroyAllWindows()

Read a new frame:  True
startseq egg on pan endseq
Read a new frame:  True
startseq egg on pan endseq
Read a new frame:  True
startseq egg on pan endseq


KeyboardInterrupt: ignored

In [0]:


vidcap = cv2.VideoCapture('testing.mp4')
fps = vidcap.get(cv2.CAP_PROP_FPS)
success,image = vidcap.read()
count = 0
success = True

    while success:
        success,frame = vidcap.read()
        count+=1
        print("time stamp current frame:",count/fps)